<a href="https://colab.research.google.com/github/willdphan/frozen-lake-dqn/blob/master/Frozen_Lake_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frozen Lake DQN
> ## Steps to Frozen Lake DQN:

>>[How Frozen Lake Works](#scrollTo=FjGSWK46cmdp)

>>[Table Dictionary](#scrollTo=ZXB0VSq4pY6u)

>>>[1. Install Libraries](#scrollTo=SEOQXocOcmdq)

>>>[2. Create the Environment](#scrollTo=JJxctuPncmdq)

>>>[3. Create the Q-table](#scrollTo=lMAviw6Ycmdr)

>>>[4. Initialize Q-learning Parameters](#scrollTo=SBPxURG-cmdr)

>>>[5. Build the Q-learning Algorithm](#scrollTo=mwm6JljVcmdr)

>>>[6. Examine the Rewards](#scrollTo=7n6TL-nlcmdr)

>>>[7. Interpret Training Results](#scrollTo=NVqaAAg0cmds)

>>>[8. Build the Q-learning Interface](#scrollTo=TGOCqkeVcmds)



### How Frozen Lake Works

This grid is our environment where `S` is the agent's starting point, and it's safe. `F` represents the frozen surface and is also safe. `H` represents a hole, and if our agent steps in a hole in the middle of a frozen lake, well, that's not good. Finally, `G` represents the goal, which is the space on the grid where the prized frisbee is located.

The agent can navigate left, right, up, and down, and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of one if it reaches the goal, and zero otherwise.

### Table Dictionary

<table>
<thead>
  <tr>
    <th>State</th>
    <th>Description</th>
    <th>Reward</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>S</td>
    <td>Agent's starting point - safe</td>
    <td>0</td>
  </tr>
  <tr>
    <td>F</td>
    <td>Frozen surface - safe</td>
    <td>0</td>
  </tr>
  <tr>
    <td>H</td>
    <td>Hole - game over</td>
    <td>0</td>
  </tr>
  <tr>
    <td>G</td>
    <td>Goal - game over</td>
    <td>1</td>
  </tr>
</tbody>
</table>

### 1. Install Libraries

In [7]:
import random
import time

import numpy as np
import gym

from IPython.display import clear_output

### 2. Create the Environment

In [8]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### 3. Create the Q-table
Retrieve the size of the space and initialize the q-table with the sizes.

`env.action_space.n` returns the number of possible actions in the environment. In the case of CartPole-v1, there are two possible actions:

1.   more right
2.   move left
3.   move down
4.   move up

Therefore, action_space_size will be equal to 4.

`env.observation_space.n` returns the number of possible states in the environment. This is continuous.

In [9]:
action_space_size = env.action_space.n # = 4 actions
state_space_size = env.observation_space.n # = 16 states, 16 tiles
action_space_size, state_space_size

(4, 16)

In [10]:
# q_table takes in state space size and action space size
q_table = np.zeros((state_space_size, action_space_size))
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

### 4. Initialize Q-learning Parameters

Define and initialize the parameters used in the Q-learning algorithm for training an agent in the CartPole-v1 environment.

In [11]:
# 1000 episodes
num_episodes = int(1e4)
# The maximum number of steps allowed per episode. If the agent reaches this limit
# without completing the task (balancing the pole), the episode will end.
max_steps_per_episode = 100

# rate at which agents update Q-values. Low rate of 0.1. Small updates.
lr = 1e-1
# determines the importance of future rewards compared to immediate rewards.
# It allows the agent to prioritize long-term rewards over short-term rewards.
dr = 0.99

# probability of agent choosing a random action instead of exploiting learned Q-values.
# Initially set to 1, agent explores the environment extensively to gather information.
# As training progresses, exploration rate decays, agent relies more on learned Q-values for decisions.
exploration_rate = 1
# upperbound for exploration rate
max_exp_rate = 1
# lowerbound for exploration rate
min_exp_rate = 1e-3
# rate at which the exploration rate decays over time. It controls the rate of
# reduction in the exploration rate as the agent gains more experience.
exp_decay_rate = 1e-3

# building a reward list for all the episodes
rewards_all_episodes = []

### 5. Build the Q-learning Algorithm

The code trains an agent using the Q-learning algorithm in the CartPole-v1 environment. It runs a loop for each episode, where the agent takes actions based on an epsilon-greedy strategy. The Q-table is updated using the Bellman equation, combining immediate rewards with expected future rewards. The exploration rate gradually decreases over episodes. The total rewards for each episode are stored for analysis. The goal is to train the agent to balance the pole on the cart for as long as possible.
TLDR;
1.    The code executes a loop for each episode, where an episode represents a complete playthrough of the game. `rewards_current_episode`, is initialized to keep track of the cumulative rewards obtained in the current episode.
2.    Within each episode, a loop runs for each step the agent takes.
  *   Choose an action based on the epsilon-greedy strategy: exploit the maximum. Q-value or explore by selecting a random action.
  *   Take the chosen action and receive feedback: new state, reward, done flag.
  *   Update the Q-table based on the Bellman equation, combining old Q-values with learned values.
  *   Transition to the new state and add the immediate reward to cumulative rewards
  *   Check if the episode is done, and if so, break the loop for the current episode and move to the next episode. Otherwise, transition to the next time-step within the episode
3. Then, we use the Exploration Rate decay to decrease the exploraion rate over episodes and keep track of the total rewards to keep track of the agent's performance.



In [12]:
# Q-learning algorithm
# for-loop for each episode
for episode in range(num_episodes):

    # reset the movements in the env
    state = env.reset()
    # check if the agent reaches the target
    done = False
    # variable for expected return G_t
    rewards_current_episode = 0

    # for loop for each step for the agent
    for step in range(max_steps_per_episode):
        # apply epsilon greedy stategy. generates a random floating-point number between 0 and 1.
        random_number = random.uniform(0, 1)
        # Exploration Vs. Exploitation trade-off. Used in the epsilon-greedy strategy
        if random_number > exploration_rate:
            # start exploitation -> getting the maximum Q-value from the possible movements of his current state.
            action = np.argmax(q_table[state, :])
        else:
            # start exploration -> select any random action to explore a random state.
            action = env.action_space.sample()

        # after taking the action, we're going to update our agent with the new
          # current state, rewards, if it reached the end or not, info
        new_state, reward, done, info = env.step(action)

        # Update our Q-table for Q(s, a) using Bellman Equation
                                  # weight of lr  * Old Q-value
        q_table[state, action] = (1 - lr) * q_table[state, action] + lr * (reward + dr*(np.max(q_table[new_state, :])))
                            # lr * (immediate reward + dsicount rate(maximum Q-value among all possible actions in the new state))
                            # AKA learned value

        # transition to the next state
        state = new_state
        # accumulates the rewards obtained by the agent in the current episode.
        rewards_current_episode += reward

        # check to see if our last action ended the episode for us,
        # meaning, did our agent step in a hole or reach the goal?
        if done:
            break
        # If the action did end the episode, then we jump out of this loop and move on to the next episode.
        # Otherwise, we transition to the next time-step.

    # Exploration Rate Decay, used to decrease the exploration rate over episodes.
    # https://en.wikipedia.org/wiki/Exponential_decay
    exploration_rate = min_exp_rate + (max_exp_rate - min_exp_rate) * np.exp(-exp_decay_rate * episode)

    # append the current rewards in the list of rewards.
    # allows for the analysis and evaluation of the agent's performance over multiple episodes
    rewards_all_episodes.append(rewards_current_episode)

### 6. Examine the Rewards
From the printout we can notice that our average reward per thousand epoisodes did indeed progress overtime. When the algorithm first start training, the first thousands episodes only average a reward of `0.062`, but by the time it got to its last thousand episodes, the reward improved to `0.74`.

Calculate the average reward per thousand episodes
`np.split` splits the NumPy array of rewards into sub-arrays of equal size, where each sub-array which represents the cumulative rewards obtained over a thousand episodes. `np.array` converts to np for easier manipulation. array calculates the number of groups (or splits) based on the total `num_episodes / 1000`. This determines the number of sub-arrays to create.

Print an output that displays the average rewards per thousand episodes for each group in the `rewards_per_thousand_episodes` list. The count number indicates the range of episodes covered in each group, and the average reward represents the average reward obtained per episode within that range.

In [13]:
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

print("Average rewards per thousand episodes".center(100, '*'))
for reward in rewards_per_thousand_episodes:
    print(f'Count No. {count:,}: {sum(reward/1000)}')
    count += 1000

*******************************Average rewards per thousand episodes********************************
Count No. 1,000: 0.04900000000000004
Count No. 2,000: 0.21900000000000017
Count No. 3,000: 0.4070000000000003
Count No. 4,000: 0.5950000000000004
Count No. 5,000: 0.6560000000000005
Count No. 6,000: 0.6980000000000005
Count No. 7,000: 0.6760000000000005
Count No. 8,000: 0.7270000000000005
Count No. 9,000: 0.7130000000000005
Count No. 10,000: 0.7430000000000005


### 7. Interpret Training Results

Our agent played `10,000` episodes. At each time step within an episode, the agent received a reward of `1` if it reached the frisbee, otherwise, it received a reward of `0`. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

So, that means for each episode, the total reward received by the agent for the entire episode is either `1` or `0`. So, for the first thousand episodes, we can interpret this score as meaning that  **6%** of the time, the agent received a reward of `1` and won the episode. And by the last thousand episodes from a total of 10,000, the agent was winning **74%** of the time.

The numbers within the square brackets represent the Q-values for the respective actions that can be taken in that state.
For example, the first row `[0.58230461 0.48825684 0.48921769 0.49125568] `represents the Q-values for the four possible actions `(0, 1, 2, and 3)` in the first state.

Higher Q-values generally indicate a higher expected cumulative reward for taking that action in the given state.

In some rows, all Q-values are zero, indicating that the agent has not yet explored or learned about the rewards associated with those state-action pairs.

As the agent interacts with the environment and learns through the Q-learning algorithm, the Q-values are updated based on observed rewards and the agent's exploration-exploitation strategy.

In [14]:
# visualize the Q-Table below
print("Q-Table".center(100, '*'))
print()

for row in q_table:
    print(' '* 25, row)

**********************************************Q-Table***********************************************

                          [0.58230461 0.48825684 0.48921769 0.49125568]
                          [0.29350113 0.25999836 0.28457958 0.44168369]
                          [0.39210816 0.20972835 0.24604732 0.27011584]
                          [0.05790049 0.10516014 0.03315454 0.0606654 ]
                          [0.60679958 0.46834759 0.43509862 0.30892254]
                          [0. 0. 0. 0.]
                          [0.31120102 0.16868329 0.12795261 0.13501385]
                          [0. 0. 0. 0.]
                          [0.37516465 0.43596654 0.37695086 0.6627882 ]
                          [0.46724687 0.71528521 0.35877242 0.39663325]
                          [0.71508712 0.34140213 0.36528914 0.26098435]
                          [0. 0. 0. 0.]
                          [0. 0. 0. 0.]
                          [0.45429822 0.49409317 0.78671695 0.48190456]
                  

### 8. Build the Q-learning Interface

Let's see how interactively the agent plays Frozen Lake. See the agent's interactions with the environment, the environment's visual display, and messages indicating whether the agent reached the goal or fell through a hole. The code provides a step-by-step visualization of the agent's behavior in the environment.


In [15]:
for episode in range(5):
    state = env.reset()
    done = False
    print(f'Episode: {episode+1}'.center(50, '='))
    time.sleep(1)

    for step in range(max_steps_per_episode):
        # reset env and indicate episode is not complete
        # for clearning the board
        clear_output(wait=True)
        # allows you to check the agent's environment
        env.render()
        time.sleep(0.4)

        # invoke the action (state) with the highest Q-value from the Q-Table for the current state
        action = np.argmax(q_table[state, :])

        # take the action and move to the new state
        new_state, reward, done, info = env.step(action)

        # acting condition
        # Checks if the episode is completed. If done is True, the agent has either reached the goal or fallen through a hole.
        if done:
            # Clears the output in the console.
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Pauses the execution for 3 seconds to display the final state and reward message.
                print('You reach the goal!'.center(50, '*'))
                time.sleep(3)
            else:
                # Pauses the execution for 3 seconds to display the final state and reward message.
                print('You fall through a hole!'.center(50, '-'))
                time.sleep(3)
                # Clears the output in the console.
                clear_output(wait=True)
            break

        # select the new state based on the agent action
        state = new_state

# close the environment
env.close()

-------------You fall through a hole!-------------
